### Validation of Bayt-Level Semantic Annotations (YaarAI)

This notebook validates **bayt-level semantic annotations** used in YaarAI,
including:

- `bayt_hint`: a short noun phrase describing what happens in the bayt
- `affect`: up to two affect labels drawn from a fixed vocabulary

The goals of this notebook are to:
- verify schema correctness,
- inspect annotation distributions,
- validate repairs and normalizations,
- perform random qualitative checks with full bayt context.

This notebook performs no regeneration.

In [1]:
import json
import random
import re
from pathlib import Path
from collections import Counter, defaultdict

### Load raw bayt text

Raw bayt text is loaded once and reused throughout the notebook
to provide full poetic context during inspection.

In [2]:
RAW_PATH = Path("../data/raw/ghazals_with_insight.jsonl")

raw_by_key = {}
raw_by_poem = defaultdict(list)

with RAW_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        r = json.loads(line)
        key = (r["poem_id"], r["bayt_id"])
        raw_by_key[key] = r["text"]
        raw_by_poem[r["poem_id"]].append(r)

len(raw_by_key)

4192

### Why multiple bayt annotation versions exist

Bayt-level semantic annotations in YaarAI were refined through a
**multi-stage, quality-driven process**.

Rather than overwriting results, each refinement produced a new version
to ensure transparency, reproducibility, and auditability.

The annotation versions are:

- **v1 — Initial extraction**
  First-pass GPT-4.1 extraction of:
  - `bayt_hint`: a short noun phrase describing what happens in the bayt
  - `affect`: up to two affect labels from a fixed vocabulary

- **v1.1 — Conciseness & drift repair**
  An automated repair pass applied to bayt hints that exceeded a length
  threshold or showed interpretive / explanatory drift.
  This step improved abstraction while preserving meaning.

- **v1.2 — Directive normalization (explicit)**
  Deterministic, rule-based removal of explicit directive framing such as:
  - «دعوت به …»
  - «توصیه به …»
  - «تشویق به …»

- **v1.3 — Directive normalization (residual)**
  Final normalization pass removing residual *soft directive* prefixes
  left behind after earlier repairs, including:
  - «توجه به …»
  - «بهره‌گیری از …»

  This step completes the transition to **fully descriptive,
  non-directive bayt hints**, without regeneration or semantic reinterpretation.

This notebook validates the **final canonical version (v1.3)**.
Earlier versions are retained for reference and provenance.

In [3]:
ANNOTATION_PATH = Path("../data/annotations/bayt_annotations_v1_3.jsonl")

rows = []
with ANNOTATION_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))

len(rows)

4192

### What is validated in this notebook

This notebook validates the **final canonical bayt annotation layer (v1.3)**.

The following aspects are checked:

- schema correctness (`poem_id`, `bayt_id`, `bayt_hint`, `affect`)
- conciseness of `bayt_hint` (noun phrase, not prose)
- absence of directive or verbal framing
- reasonable distribution of affect labels
- semantic alignment between bayt text, hint, and affect

All checks are **diagnostic and qualitative**.
No further annotation, regeneration, or normalization is performed here.

In [4]:
for r in rows:
    assert isinstance(r["poem_id"], int)
    assert isinstance(r["bayt_id"], int)
    assert isinstance(r["bayt_hint"], str)
    assert r["bayt_hint"].strip()
    assert isinstance(r["affect"], list)
    assert len(r["affect"]) <= 2

### Bayt hint length statistics

Bayt hints should remain short noun phrases, not explanatory prose.

In [5]:
lengths = [len(r["bayt_hint"].split()) for r in rows]

min(lengths), max(lengths), round(sum(lengths) / len(lengths), 2)

(1, 9, 5.2)

### Inspection of long bayt hints

Bayt hints exceeding a length threshold are inspected manually
to detect interpretive drift.

In [6]:
long_hints = [r for r in rows if len(r["bayt_hint"].split()) >= 8]
len(long_hints)

246

In [7]:
for r in long_hints:
    key = (r["poem_id"], r["bayt_id"])
    print(f"poem_id={r['poem_id']} bayt_id={r['bayt_id']}")
    print("BAYT:", raw_by_key.get(key))
    print("HINT:", r["bayt_hint"])
    print("-" * 80)

poem_id=3 bayt_id=2
BAYT: بده ساقی می باقی که در جنت نخواهی یافت / کنار آب رکناباد و گل‌گشت مصلا را
HINT: درخواست می باقی در کنار آب و گل‌گشت
--------------------------------------------------------------------------------
poem_id=5 bayt_id=6
BAYT: آسایش دو گیتی تفسیر این دو حرف است / با دوستان مروت با دشمنان مدارا
HINT: پند اخلاقی درباره رفتار با دوستان و دشمنان
--------------------------------------------------------------------------------
poem_id=5 bayt_id=10
BAYT: سرکش مشو که چون شمع از غیرتت بسوزد / دلبر که در کف او موم است سنگ خارا
HINT: هشدار به سرکشی عاشق در برابر دلبر قدرتمند
--------------------------------------------------------------------------------
poem_id=5 bayt_id=12
BAYT: خوبان پارسی‌گو بخشندگان عمرند / ساقی بده بشارت رندان پارسا را
HINT: بخشش عمر از سوی خوبان به رندان پارسا
--------------------------------------------------------------------------------
poem_id=6 bayt_id=3
BAYT: مژه‌ی سیاهت ار کرد به خون ما اشارت / ز فریب او بیندیش و غلط مکن، نگارا
HINT: اشارت مژه 

### Affect label distribution

We inspect the frequency of affect labels to ensure reasonable coverage
and detect skew or collapse.

In [8]:
affects = [a for r in rows for a in r["affect"]]
Counter(affects)

Counter({'حسرت': 1483,
         'شوق': 1251,
         'امید': 752,
         'اندوه': 751,
         'بی\u200cقراری': 538,
         'آرامش': 450,
         'حیرت': 398,
         'ناامیدی': 113})

### Affect cardinality check

Each bayt may have 0, 1, or 2 affect labels.

In [9]:
Counter(len(r["affect"]) for r in rows)

Counter({1: 2056, 2: 1840, 0: 296})

### Random global inspection

A random sample of bayts is inspected to verify overall annotation quality.

In [10]:
random.seed(42)

for r in random.sample(rows, 10):
    key = (r["poem_id"], r["bayt_id"])
    print(f"poem_id={r['poem_id']} bayt_id={r['bayt_id']}")
    print("BAYT:", raw_by_key[key])
    print("HINT:", r["bayt_hint"])
    print("AFFECT:", r["affect"])
    print("-" * 80)

poem_id=110 bayt_id=6
BAYT: بس تجربه کردیم در این دیر مکافات / با دردکشان هر که درافتاد برافتاد
HINT: آزمون دشواری‌ها و سرنوشت دردکشان
AFFECT: ['اندوه']
--------------------------------------------------------------------------------
poem_id=24 bayt_id=3
BAYT: می بده تا دهمت آگهی از سر قضا / که به روی که شدم عاشق و از بوی که مست
HINT: درخواست می و بیان راز عاشقی
AFFECT: ['شوق']
--------------------------------------------------------------------------------
poem_id=269 bayt_id=7
BAYT: هوای مسکن مالوف و عهد یار قدیم / ز رهروان سفرکرده، عذرخواهت بس
HINT: یاد خانه آشنا و عهد دوست قدیم
AFFECT: ['حسرت']
--------------------------------------------------------------------------------
poem_id=241 bayt_id=4
BAYT: چو در میان مراد آورید دست امید / ز عهد صحبت ما در میانه یاد آرید
HINT: یادآوری عهد دوستی
AFFECT: ['امید', 'حسرت']
--------------------------------------------------------------------------------
poem_id=219 bayt_id=4
BAYT: شد از خروج ریاحین چو آسمان روشن- / زمین، به اختر میمون و طالع 

### Inspection of normalized bayt hints

Bayt hints modified by rule-based normalization are inspected
to ensure semantic content was preserved.

In [11]:
normalized = [
    r for r in rows
    if r.get("annotation_meta", {}).get("manual_norm") is True
]

len(normalized)

186

In [12]:
random.seed(42)

for r in random.sample(normalized, 10):
    key = (r["poem_id"], r["bayt_id"])
    print(f"poem_id={r['poem_id']} bayt_id={r['bayt_id']}")
    print("BAYT:", raw_by_key[key])
    print("HINT:", r["bayt_hint"])
    print("-" * 80)

poem_id=473 bayt_id=1
BAYT: وقت را غنیمت دان آن قدر که بتوانی / حاصل از حیات ای جان این دم است تا دانی
HINT: زمان
--------------------------------------------------------------------------------
poem_id=121 bayt_id=6
BAYT: چو بر روی زمین باشی، توانایی، غنیمت دان / که دوران، ناتوانی‌ها، بسی زیر زمین دارد
HINT: فرصت زندگی
--------------------------------------------------------------------------------
poem_id=29 bayt_id=7
BAYT: سبز است در و دشت بیا تا نگذاریم / دست از سر آبی که جهان جمله سراب است
HINT: لذت‌های زودگذر طبیعت
--------------------------------------------------------------------------------
poem_id=250 bayt_id=9
BAYT: حافظ اندیشه کن از نازکی خاطر یار / برو از درگهش این ناله و فریاد ببر
HINT: رعایت حال یار
--------------------------------------------------------------------------------
poem_id=239 bayt_id=4
BAYT: مکن ز غصه شکایت که در طریق طلب / به راحتی نرسید آن که زحمتی نکشید
HINT: شکیبایی در مسیر تلاش
-------------------------------------------------------------------------

### Affect-specific slice inspection

We inspect bayts sharing the same affect label to verify semantic coherence.

In [13]:
subset = [r for r in rows if "حسرت" in r["affect"]]

random.seed(123)

for r in random.sample(subset, 8):
    key = (r["poem_id"], r["bayt_id"])
    print(f"poem_id={r['poem_id']} bayt_id={r['bayt_id']}")
    print("BAYT:", raw_by_key[key])
    print("HINT:", r["bayt_hint"])
    print("AFFECT:", r["affect"])
    print("-" * 80)

poem_id=30 bayt_id=2
BAYT: تا عاشقان به بوی نسیمش دهند جان / بگشود نافه‌ای و در آرزو ببست
HINT: بوی خوش نسیم و بسته شدن در آرزو
AFFECT: ['حسرت']
--------------------------------------------------------------------------------
poem_id=179 bayt_id=7
BAYT: توانگرا! دل درویش خود به دست آور / که مخزن زر و گنج درم نخواهد ماند
HINT: جلب محبت نیازمند
AFFECT: ['حسرت']
--------------------------------------------------------------------------------
poem_id=56 bayt_id=8
BAYT: دور مجنون گذشت و نوبت ماست / هر کسی پنج روز، نوبت اوست
HINT: پایان نوبت پیشینیان و آغاز نوبت جدید
AFFECT: ['حسرت']
--------------------------------------------------------------------------------
poem_id=280 bayt_id=5
BAYT: جمال کعبه مگر عذر رهروان خواهد / که جان زنده دلان سوخت در بیابانش
HINT: رنج عاشقان در راه وصال
AFFECT: ['حسرت', 'اندوه']
--------------------------------------------------------------------------------
poem_id=179 bayt_id=2
BAYT: من ار چه در نظر یار خاک‌سار شدم / رقیب نیز چنین محترم نخواهد ماند
HINT: مقای